# L10 : Ryanodine Receptor

Calcium influx occurs via two principal pathways : inflow from the extracellular medium through calcium channels in the plasma membrane and release from internal stores. Since the inflow from the extracellular medium has been discussed in the previous lectures, here we focus on intra-cellular calcium release pathways.

Calcium release from internal storages, in cardiac cells primarily the sarcoplasmic reticulum, is the second major way for calcium to enter the cytoplasm, and is mediated principally by the ryanodine receptor (RyR) calcium channels. In this section we will focus on the RyR that got its name due to sensitivity to the plant alkoloid ryanodine which decreases the open probability of the channel by locking the RyRs at its half-open state at nanomolar concentrations, yet fully closes them at micromolar concentration. On the other hand, caffein increases the open probablility of RyRs.

RyRs play an important role in the excitation-contraction coupling (ECC) in the heart and underlies the priciple of calcium-induced calcium release (CICR), where a small amount of calcium entering the cell initiates an explosive release of calcium from the sarcoplasmic reticulum (SR). Calcium concentrations in the cell both activate and inactivate calcium release from the SR, but at different rates. 

Additionally, the cytosolic calcium concentration is controled by calcium buffering. Calcium is an important messenger ion involved in different cellular processes and therefore it has a complex transporting and buffering system. Calcium is heavily bound by large proteins in the cell with estimates of around $99\%$ of the total cytoplasmic calcium. We will discuss this later. 

As already mentioned, RyRs are sensitive to calcium and hence, a small increase of intracellular calcium concentation can activate RyRs situated in the membrane of the SR and increase the calcium flux into the cytosol through calcium triggered calcium-induced calcium release (CICR). It is also known that calcium can inactivate RyR *in vitro*, but until now it is still unknown whether such an inactivation plays an important physiological role *in vivo*.

A simple model of CICR (Friel, 1995) provides an excellent quantitative description of oscillatory behaviour in response to caffeine, or mild depolarization. Although these oscillations are dependent on external calcium, they occur at a fixed membrane potential and involve the release of calcium from the SR via ryanodine receptors. 

Due to the simplicity of the model, it is an easy example of how theory can supplement experiments, providing an interpretation of experimental results as well as quantitative predictions that can subsequently be tested.

<img src="RyR1.eps" alt="Alt text that describes the graphic" title="Title text" />

This schematic diagram of the model replicates a single intracellular calcium store (sarcomplasmic reticulum) which exchanges calcium with the cytoplasm (with fluxes $J_{L2}$ and $J_{P2}$). In return the cytoplasm exchanges calcium with the external medium via $J_{L1}$ and $J_{P1}$. Thus, writing this in equations 

\begin{equation}
\begin{split}
\frac{dc}{dt} =& J_{L1} - J_{P1} + J_{L2} - J_{P2} \\
\frac{dc_{SR}}{dt} =& \gamma (-J_{L2} + J_{P2})
\end{split}
\end{equation}

where $c$ is the cytosolic calcium concentration, and $c_{SR}$ denotes the SR calcium concentration. Also, $\gamma$ is the ratio of the cytoplasmic volume to the SR volume. This ratio is important as it gives an indication of how the flux between two compartments is modulated by its volumes. In other words, the volume together with the ion concentration describes the density of the ions within a compartment and therefore influences the flux between compartments with very different ion densities. 

The fluxes in this model are chosen in a simple way, as linear functions of the concentrations:
\begin{equation}
\begin{split}
J_{L1} =& k_1 (c_0 -c) \quad \quad &\text{calcium entry} \\
J_{P1} =& k_2c \quad \quad & \text{calcium extrusion} \\
J_{L2} =& k_3 (c_{SR} -c) \quad \quad & \text{calcium release} \\
J_{P2} =& k_4c \quad\quad & \text{calcium uptake}
\end{split}
\end{equation}

where $c_0$ is the external calcium concentration which is assumed to be fixed (as it is often done in experiments). Depolarization is modeled by an increase of $k_1$, the rate of calcium entry from the outside. Note that calcium entry and calcium release are gradient based diffusion equations, while the calcium extrusion and calcium release are simplified pump equations. 

An increase of the rate of calcium release from the internal store (e.g. by applying caffein) can be modeled by an increase in $k_3$.

To model CICR in a simple way, $k_3$ is implemented as an increasing sigmoidal function of c, i.e. release is modelled with Hill type dynamics. 

\begin{equation}
\begin{split}
k_3 &= \kappa_1 + \frac{\kappa_2c^n}{K_d^n+c^n}\\
J_{L2} &= k_3(c_{SR} - c) \\
& = (\kappa_1 + \frac{\kappa_2c^n}{K_d^n+c^n})(c_{SR}-c)
\end{split}
\end{equation}

The parameters are determined by fitting this nonlinear model to the time course of an oscillation.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.integrate

from array import array
from pylab import *
from scipy.integrate.odepack import odeint

from scipy.integrate import odeint
from math import exp, log, sqrt, pi


def CICR(y, t):
    #constants
    k_1 = 2*10**(-5);
    k_2 = 0.13;
    k_4 = 0.9;
    kappa_1 = 0.013;
    kappa_2 = 0.58;
    K_d = 0.5;
    n = 3;
    gamma = 4.17;
    c0 = 1000;

    #input
    c = y[0];
    cSR = y[1];

    #calculation of k_3
    k_3 = kappa_1 + (kappa_2*c**n)/(K_d**n+c**n);

    #calcium entry
    J_L1 = k_1 * (c0 - c)

    #calcium extrusion
    J_P1 = k_2 * c;

    #calcium release
    J_L2  = k_3 *(cSR - c);

    #calcium uptake
    J_P2 = k_4 *c;

    #caculate time dependent functions
    ydot = [(J_L1-J_P1+J_L2-J_P2) , gamma*(-J_L2+J_P2)]; #dc #dcSR

    return ydot


#initial conditions
y0 = [0.0795,4.1725];
time = np.linspace(0.0,10.0, num = 1000)

#ode function call
y = scipy.integrate.odeint(CICR, y0, time);

Even though we have an implementation for CICR, we still do not have an explicit formulation for the RyR opening an<font color=green>d</font> closing dependent on the calcium concentration.

## A 4-State Model of the RyR

Calcium-induced calcium release as described above is an event triggered via opening of RyRs. The major stimulus for CICR is calcium entering the cell via sarcolemmal L-type channels. The rate and amount of calcium release from the SR is tightly controlled by the magnitude and duration of L-type calcium current. CICR is a very local control system, during which calcium enters the cell via the L-type channel increasing the calcium concentration in the dyadic space where the RyR are located in the SR membrane. Thus, this local control of macroscopic SR calcium release is archieved by graded statistical recruitment of individual, autonomous, stochastic release events. 

Depending on the species, from a few tens to approximately 200 RyRs are clustered in a two dimensional crystal-lattice array on the surface of SR release terminals, across the 15nm cleft of te dyad junction between the cell membrane and the SR. In  mathematical models, the activation of the RyRs is simulated by stochastic interactions via calcium entering in the dyadic cleft through the L-Type channel. To represent the stochastic nature of RyRs, usually a Markov model approach is used. 

Usually, the set of channels at each dyad is treated as a single stochastic system whose state is defined by specifying the Markov state of each of the individual channels. State transitions of individual channels are considered to take place instantaneaously so that a macrostate transition corresponds to a transition of exactly one channel. 

<img src="RyR2.eps" alt="Alt text that describes the graphic" title="Title text" />




Using a phenomenological formulation (Stern et al.,1999), the RyRs are represented, in simplest form, by the combination of activation (binding of two calcium ions) and calcium dependent inactivation. The Markov model is specified by 4 different states
- R : no calcium ions attached, resting state
- O : calcium is attached to activating site, open state
- RI : calcium is attached to inactivating site, inactive state
- I : calcium attached to both sites (activating and inactivating site), inactive state

As described above, to model the state of the RyR cluster, we calculate the fraction of RyRs in the different states:

\begin{equation}
\begin{split}
x_R &= \frac{R}{RyR_{tot}}\\
x_O &= \frac{O}{RyR_{tot}}\\
x_I &= \frac{I}{RyR_{tot}} \\
x_{RI} &= \frac{RI}{RyR_{tot}} \\
&= 1-x_R-x_O-x_I\\
\end{split}
\end{equation}

In addition to a detailed, probabilistic formulation for the RyR, the models for the pumps are updated:

\begin{equation}
\begin{split}
J_P = V_{max} \frac{c^2}{K^2 + c^2}
\end{split}
\end{equation}

From the model scheme, we get the following model equations

\begin{equation}
\begin{split}
\frac{dx_R}{dt} &= K_{om}\cdot x_O + K_m\cdot x_{RI}-(K_0SRCa + K_ISRCa)\cdot x_R\cdot c\\
\frac{dx_O}{dt} &= K_{im}\cdot x_I - K_{om}\cdot x_{O}+(K_0SRCa\cdot x_R - K_ISRCa\cdot x_I)\cdot c\\
\frac{dx_I}{dt} &= -K_{om}\cdot x_I - K_{im}\cdot x_{I}+(K_0SRCa\cdot x_{RI} + K_ISRCa\cdot x_O)\cdot c\\
\frac{dc}{dt} =& J_{L1} - J_{P1} + J_{L2} - J_{P2} \\
\frac{dc_{SR}}{dt} =& \gamma (-J_{L2} + J_{P2})\\
J_{L1} =& k_1 (c_0 -c)  \\ 
J_{P1} =& \frac{q_{1_1}c}{q_{2_1}^2+c^2} \\
J_{L2} =& k_3 (c_{SR} -c)  + \color{red}{k_{RyR}x_O}\\
J_{P2} =& \frac{q_{1_2}c}{q_{2_2}^2+c^2}
\end{split}
\end{equation}

These ODEs can be solved for a certain duration to see how the RyR channels behave. Implementing a formulation like this in a cardiac cell model provides a more detailed description of the calcium handling processes.

## Summary : Ryanodine Receptor

1. Sits at the surface of intra-cellular calcium stores
   - Endoplasmic Reticulum (ER)
   - Sarcoplasmic Reticulum (SR)
2. Sensitive to calcium for both activation and inactivation
3. Upon stimulation<font color=green>,</font> calcium is released from the stores
4. Two different pathways
   - Triggering from action potential through extra cellular calcium flow
   - Calcium oscillations observed in some neurons at fixed membrane potentials


        

## References

- Stern, Michael D., et al. "Local control models of cardiac excitation–contraction coupling: a possible role for allosteric interactions between ryanodine receptors." The Journal of general physiology 113.3 (1999): 469-489.
- Shannon, Thomas R., et al. "A mathematical treatment of integrated Ca dynamics within the ventricular myocyte." Biophysical journal 87.5 (2004): 3351-3371.
- "Numerical Solution of the Transient Diffusion Equation Using the Finite Difference (FD) Method." CE30125-Lecture11
- James Keener and James Sneyd "Mathematical Physiology I: Cellular Physiology." (Second Edition) 